<a href="https://colab.research.google.com/github/MikhailMatytsin/dec-python-course/blob/main/Gross_up_prototype_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here's a Python translation of your Stata code using pandas and numpy. This version attempts to preserve the structure and logic of your original model, with clear comments to help map concepts from Stata to Python. The code assumes you're working with a single observation (as in your set obs 1) and uses iterative adjustments to align statutory and net incomes.



In [16]:
import pandas as pd
import numpy as np

# PARAMETERS FOR SIMULATIONS
pit_rate_b = 0.13
sic_rate_b = 0.3

pit_rate_r = 0.13
sic_rate_r = 0.3

pit_pt_b = 1
sic_pt_b = 1

pit_pt_r = 1
sic_pt_r = 1

market_income = ["labor_inc", "self_inc", "other_inc"]
direct_taxes = ["pit"]
SSC = ["sic"]

d = 10**-8
report = 1

# INITIAL DATA SETUP
df = pd.DataFrame({'hh_id': [1], 'p_id': [1], 'labor_inc': [87], 'self_inc': [35], 'other_inc': [50]})

# Step 1: Finding statutory wages
for inc in market_income:
    df[f'{inc}_orig'] = df[inc]
    df[f'{inc}_stat'] = df[inc]

sic_rate = sic_rate_b
pit_rate = pit_rate_b

sic_pt = sic_pt_b
pit_pt = pit_pt_b

# Iterative adjustment of statutory income
max_gap = d * 2
min_gap = 0
s = 1

def SSC_direct_taxes_statutory(df, pit_taxable_list, sic_taxable_list):
    for tax, rate, pt, taxable_list in [
        ('pit', pit_rate, pit_pt, pit_taxable_list),
        ('sic', sic_rate, sic_pt, sic_taxable_list)
    ]:
        df[f'{tax}_base'] = 0
        for inc in taxable_list:
            df[f'{tax}_base'] += df[f'{inc}_stat']

        for inc in taxable_list:
            with np.errstate(divide='ignore', invalid='ignore'):
                df[f'{tax}_sh_{inc}'] = np.where(
                    df[f'{tax}_base'] != 0,
                    df[f'{inc}_stat'] / df[f'{tax}_base'],
                    0
                )

        for inc in market_income:
            if f'{tax}_sh_{inc}' not in df.columns:
                df[f'{tax}_sh_{inc}'] = 0

        df[f'{tax}_stat'] = -1 * rate * df[f'{tax}_base']
        df[tax] = df[f'{tax}_stat'] * pt

while max_gap > d or min_gap < -d:
    SSC_direct_taxes_statutory(df, ['labor_inc', 'self_inc'], ['labor_inc'])

    for inc in market_income:
        df[f'{inc}_net_it'] = df[f'{inc}_stat']
        for tax in direct_taxes:
            df[f'{inc}_net_it'] += df[f'{tax}_sh_{inc}'] * df[f'{tax}_stat']

        df[f'{inc}_gap'] = df[f'{inc}_orig'] - df[f'{inc}_net_it']

    if s % report == 0:
        print(f"Step {s}")
        print(df[[f'{inc}_gap' for inc in market_income]].describe())

    max_gap = max(df[f'{inc}_gap'].max() for inc in market_income)
    min_gap = min(df[f'{inc}_gap'].min() for inc in market_income)

    for inc in market_income:
        df[f'{inc}_stat'] += df[f'{inc}_gap']

    s += 1

print(f"End at step {s}")
print(df.columns)
#print(df[[f'{inc}_gap' for inc in market_income]].describe())

# Step 2: Calculate equilibrium income
for inc in market_income:
    df[inc] = df[f'{inc}_orig']
    for tax in SSC + direct_taxes:
        df[inc] -= df[f'{tax}_sh_{inc}'] * df[tax]

# Store results for prototype
for inc in market_income:
    df[f'{inc}_net_b'] = df[f'{inc}_orig']
    df[f'{inc}_eq_b'] = df[inc]
    df[f'{inc}_stat_b'] = df[f'{inc}_stat']

for tax in SSC + direct_taxes:
    df[f'{tax}_b'] = df[tax]
    df[f'{tax}_stat_b'] = df[f'{tax}_stat']
    df[f'{tax}_base_b'] = df[f'{tax}_base']
    for inc in market_income:
        df[f'{tax}_sh_{inc}_b'] = df[f'{tax}_sh_{inc}']

print(df)

Step 1
       labor_inc_gap  self_inc_gap  other_inc_gap
count           1.00          1.00            1.0
mean           11.31          4.55            0.0
std              NaN           NaN            NaN
min            11.31          4.55            0.0
25%            11.31          4.55            0.0
50%            11.31          4.55            0.0
75%            11.31          4.55            0.0
max            11.31          4.55            0.0
Step 2
       labor_inc_gap  self_inc_gap  other_inc_gap
count         1.0000        1.0000            1.0
mean          1.4703        0.5915            0.0
std              NaN           NaN            NaN
min           1.4703        0.5915            0.0
25%           1.4703        0.5915            0.0
50%           1.4703        0.5915            0.0
75%           1.4703        0.5915            0.0
max           1.4703        0.5915            0.0
Step 3
       labor_inc_gap  self_inc_gap  other_inc_gap
count       1.000000      1.0

**Notes:**


* The translation assumes a single-row DataFrame, as in your Stata example.
* The compute_taxes() function mimics the Stata program SSC_direct_taxes_statutory.
* Handling of missing values is simplified — in production, ensure NaNs are handled robustly.
* The code structure supports extension to multiple households if needed by expanding the DataFrame rows.

Let me know if you want to modularize this into functions or extend it to support vectorized reform simulations.